<a href="https://colab.research.google.com/github/kassymoto/daigaku/blob/master/daigaku_arekore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import requests
from bs4 import BeautifulSoup, element
from datetime import datetime
import pandas as pd

In [0]:
# 大学の情報を格納するクラス
class University:
  # 検索一覧から取得できる情報
  name = ''  # 大学名
  page_url = ''  # 大学情報ページのURL soupで保持
  hensa_upper = '' # 大学のUpper偏差値
  hensa_lower = '' # 大学のLower偏差値
  prefecture = '' # 大学の所在都道府県
  manage = ''  # 私立・国立など
  
  # 個別ページから取得する情報
  star = 0  # レビューの星の数
  review_num = 0  # 口コミ件数
  review_upper = []  # 口コミ（評判）のやつ。カテゴリ、得点、順位で1ペア
  gakumon = []  # 学べる学問
  gakuhi = []  # カテゴリ、入学金、年間授業料で1ペア
  station = ''  # 最寄り駅
  gakubu = []  # 学部
  rank = []  # 口コミの順位
  
  # 検索一覧から取得できる情報
  def __init__(self, name, url, hensa, prefecture, manage):
    self.name = name
    self.page_url = requests.get(url)
    self.edit_hensa(hensa)
    self.prefecture = prefecture
    self.manage = manage
    
  # 個別ページから取得する情報
  def import_from_individual_page(self, star, review_num, review_upper, gakumon, gakuhi, station, gakubu, rank):
    self.star = star
    self.review_num = review_num
    self.review_upper =review_upper
    self.gakumon = gakumon
    self.gakuhi = gakuhi
    self.station = station
    self.gakubu = gakubu
    self.rank = rank
  
  # 偏差値が　Low - Up表記なので、これをhensa_upperとhensa_lowerに分ける
  def edit_hensa(self, hensa):
    hensa = hensa.split()
    
    if len(hensa)==1:
      if hensa[0] == '-':
        self.hensa_upper = 'None'
        self.hensa_lower = 'None'
      
      elif hensa[0] == 'BF':
        self.hensa_upper = 0
        self.hensa_lower = 0
      
      else:
        self.hensa_upper = float(hensa[0])
        self.hensa_lower = float(hensa[0])
     
    else:
      self.hensa_upper = float(hensa[-1])
      self.hensa_lower = float(hensa[0]) if hensa[0] != 'BF' else 0
  
  # デバッグ用プリンタ
  def printer(self):

    s = f'''
    大学名：{self.name}
    偏差値：{self.hensa_lower} ～ {self.hensa_upper}
    都道府県：{self.prefecture}
    形態：{self.manage}
    評価：{self.star}
    口コミ件数：{self.review_num}
    口コミ（評判）：
    {self.review_upper}
    学べる学問：
    {self.gakumon}
    学費：
    {self.gakuhi}
    最寄り駅：{self.station}
    学部：{self.gakubu}
    口コミ順位：{self.rank}
    '''
    print(s)

In [0]:
# urlを投げて、ページ番号指定してぬく
def get_unv_info(url, page_num):
  global unv_list
  
  mdj_url = requests.get(url + f'page={page_num}/')
  
  # みんなの大学情報のsoupを作成
  mdj_soup = BeautifulSoup(mdj_url.content, 'html.parser')
  
  # 名前、県、私立・国立、URLが含まれる
  unv_name_all = mdj_soup.find_all("div", class_='mod-listSearch-name')
  unv_hensa_all = mdj_soup.find_all("div", class_='mod-listSearch-devi')

  for unv_name, unv_hensa in zip(unv_name_all, unv_hensa_all):
    # 名前と、県/マネージを取得
    name, prefecture_and_manage = unv_name.get_text().split('\n')[1:-1]

    # 大学へのリンクを取得
    url = mdj_default + unv_name.a['href']

    # 県/マネージを分ける
    prefecture, manage = prefecture_and_manage.split('/')

    # 偏差値を取得
    hensa = unv_hensa.get_text().split('\n')[3]

    unv_list.append(University(name, url, hensa, prefecture, manage))

In [0]:
def get_unv_indivisual_info(unv):
  #debug
  print(unv.name)
  
  unv_soup = BeautifulSoup(unv.page_url.content, 'html.parser')

  # レビューの星の数
  try:
    star = float(unv_soup.find('p', class_='schMod-school-point').get_text())
  except AttributeError:
    star = 0.0

  # 口コミ件数
  try:
    review_num = int(unv_soup.find('p', class_='schMod-school-num').span.get_text())
  except AttributeError:
    review_num = 0
    
  # 口コミ（評判）のタイトル
  review_upper_title = unv_soup.find_all('div', class_='mod-reviewTotalv2-item')

  # 口コミ（評判）の点数
  review_upper_score = unv_soup.find_all('div', class_='mod-reviewTotalv2-score')

  # 口コミ（評判）の順位
  review_upper_rank = unv_soup.find_all('div', class_='mod-reviewTotalv2-rank')

  # 口コミ（評判）をまとめる
  review_upper = []
  for title, score, rank in zip(review_upper_title, review_upper_score, review_upper_rank):
    title = title.get_text()
    if '-' in score.get_text():
      score = 'None'
    else:
      score = float(score.get_text())
    if rank.get_text()[:-1] == '':
      rank = 'None'
    else:
      rank = int(rank.get_text()[:-1])

    unv_dict = {}
    unv_dict['title'] = title
    unv_dict['score'] = score
    unv_dict['rank'] = rank

    review_upper.append(unv_dict)

  # 学べる学問
  gakumon_all = unv_soup.find_all('div', class_='schMod-departmentList-tag')
  gakumon_list = []
  for gakumon in gakumon_all:
    gakumon_list += gakumon.get_text().split()
  gakumon_list = [g for g in gakumon_list if g != '...']


  # 学費
  gakuhi_all = unv_soup.find_all('td', class_='tx-ac')
  gakuhi_list = []
  gakuhi_dict = {}
  for i in range(len(gakuhi_all)):
    # 文系・理系・学部名
    if i == 0 or i == 3:
      gakuhi_dict['category'] = gakuhi_all[i].get_text()

    # 入学金
    if i == 1 or i == 4:
      if '～' in gakuhi_all[i].get_text():
        gakuhi_dict['enrollment_fee_lower'] = int(gakuhi_all[i].get_text().split()[0][:-1].replace(',', ''))
        gakuhi_dict['enrollment_fee_upper'] = int(gakuhi_all[i].get_text().split()[-1][:-1].replace(',', ''))
        
      elif gakuhi_all[i].get_text() == '-':
        gakuhi_dict['enrollment_fee_lower'] = 'None'
        gakuhi_dict['enrollment_fee_upper'] = 'None'
       
      else:
        gakuhi_dict['enrollment_fee_lower'] = int(gakuhi_all[i].get_text()[:-1].replace(',', ''))
        gakuhi_dict['enrollment_fee_upper'] = int(gakuhi_all[i].get_text()[:-1].replace(',', ''))
        
    # 年間授業料は下限と上限あり
    if i == 2 or i == 5:
      if gakuhi_all[i].get_text() == '-':
        gakuhi_dict['gakuhi_lower'] = 'None'
        gakuhi_dict['gakuhi_upper'] = 'None'
      else:
        gakuhi_dict['gakuhi_lower'] = int(gakuhi_all[i].get_text().split()[0][:-1].replace(',', ''))
        gakuhi_dict['gakuhi_upper'] = int(gakuhi_all[i].get_text().split()[-1][:-1].replace(',', ''))
      gakuhi_list.append(gakuhi_dict)
      gakuhi_dict = {}

  # 最寄り駅
  station_all = unv_soup.find('p', class_='schMod-school-spec')
  station = station_all.get_text().split('/')[1].replace('\n','').replace(' ','')

  # 学部
  gakubu = []
  try:
    gakubu_all = unv_soup.find('div', class_='schMod-selectPart').find_all('a')
    for g in gakubu_all:
      gakubu.append(g.get_text())
  except AttributeError:
    pass

  # 大学の口コミ順位
  try:
    rank = int(unv_soup.find('div', class_='mod-reviewTotal-rate-review').a.em.get_text())
  except AttributeError:
    rank = 'None'
  
  unv.import_from_individual_page(star, review_num, review_upper, gakumon_list, gakuhi_list, station, gakubu, rank)

In [0]:
# みんなの大学情報のURL　全大学検索結果のURL
first_mdj_url = 'https://www.minkou.jp/university/search/all/'

# リンクの前にくっつける用
mdj_default = 'https://www.minkou.jp'

In [57]:
# 結果格納リスト
unv_list = []

# ページから基本情報を取得 41まで
for i in range(1,41):
  print(f'P.{i}を取得')
  get_unv_info(first_mdj_url, i)

P.1を取得
P.2を取得
P.3を取得
P.4を取得
P.5を取得
P.6を取得
P.7を取得
P.8を取得
P.9を取得
P.10を取得
P.11を取得
P.12を取得
P.13を取得
P.14を取得
P.15を取得
P.16を取得
P.17を取得
P.18を取得
P.19を取得
P.20を取得
P.21を取得
P.22を取得
P.23を取得
P.24を取得
P.25を取得
P.26を取得
P.27を取得
P.28を取得
P.29を取得
P.30を取得
P.31を取得
P.32を取得
P.33を取得
P.34を取得
P.35を取得
P.36を取得
P.37を取得
P.38を取得
P.39を取得
P.40を取得


各大学個別ページ内のデータを抜く  
個別ページから取得したい情報

In [90]:
for unv in unv_list:
  get_unv_indivisual_info(unv)
  unv.printer()

愛国学園大学

    大学名：愛国学園大学
    偏差値：35.0 ～ 35.0
    都道府県：千葉県
    形態：私立
    評価：2.75
    口コミ件数：4
    口コミ（評判）：
    []
    学べる学問：
    ['文化学', '生活科学', '経営情報学']
    学費：
    [{'category': '人間文化学部', 'enrollment_fee_lower': 250000, 'enrollment_fee_upper': 250000, 'gakuhi_lower': 650000, 'gakuhi_upper': 650000}]
    最寄り駅：四街道駅
    学部：['人間文化学部']
    口コミ順位：None
    
愛知医科大学

    大学名：愛知医科大学
    偏差値：52.5 ～ 65.0
    都道府県：愛知県
    形態：私立
    評価：3.92
    口コミ件数：29
    口コミ（評判）：
    [{'title': '講義・授業', 'score': 3.63, 'rank': 290}, {'title': '研究室・ゼミ', 'score': 3.03, 'rank': 537}, {'title': '就職・進学', 'score': 3.91, 'rank': 145}, {'title': 'アクセス・立地', 'score': 2.05, 'rank': 548}, {'title': '施設・設備', 'score': 3.76, 'rank': 168}, {'title': '友人・恋愛', 'score': 3.93, 'rank': 34}, {'title': '学生生活', 'score': 3.3, 'rank': 253}]
    学べる学問：
    ['医学', '看護']
    学費：
    [{'category': '医学部', 'enrollment_fee_lower': 1500000, 'enrollment_fee_upper': 1500000, 'gakuhi_lower': 3000000, 'gakuhi_upper': 3000000}, {'category': '看護学部', 'enro

Pandasに突っ込む

  name = ''  # 大学名  
  page_url = ''  # 大学情報ページのURL soupで保持  
  hensa_upper = '' # 大学のUpper偏差値  
  hensa_lower = '' # 大学のLower偏差値  
  prefecture = '' # 大学の所在都道府県  
  manage = ''  # 私立・国立など  
    
  個別ページから取得する情報  
  star = 0  # レビューの星の数  
  review_num = 0  # 口コミ件数  
  review_upper = []  # 口コミ（評判）のやつ。カテゴリ、得点、順位で1ペア  
  gakumon = []  # 学べる学問  
  gakuhi = []  # カテゴリ、入学金、年間授業料で1ペア  
  station = ''  # 最寄り駅  
  gakubu = []  # 学部  
  rank = []  # 口コミの順位  

In [0]:
# 充足率抜くやつ
import re

jusoku_url = requests.get('https://tanuki-no-suji.at.webry.info/201811/article_1.html')
jusoku_soup = BeautifulSoup(jusoku_url.content, 'html.parser')

jusoku_all = jusoku_soup.find('td', id="blog-body")

jusoku_dict = {}

for jusoku in jusoku_all.childGenerator():
  if jusoku.__class__ == element.NavigableString and '大学（' in str(jusoku):
    jusoku = re.split('[：（]', str(jusoku))
    
    jusoku_rank = int(jusoku[0][:-1])
    
    jusoku_unv_name = jusoku[1][1:]
    
    jusoku_ritsu = float(jusoku[2][:-1])
    
    jusoku_dict[jusoku_unv_name] = {
        'jusoku_rank' : jusoku_rank,
        'jusoku_ritsu' : jusoku_ritsu
    }

In [91]:
import pandas as pd

# pandas用リスト
name_list = []
hensa_lower_list = []
hensa_upper_list = []
prefecture_list = []
manage_list = []
star_list = []
review_num_list = []
gakumon_num_list = []
gakubu_num_list = []
rank_list = []
# 充足率関係
jusoku_rank_list = []
jusoku_ritsu_list = []

# Listにつっこむ
for unv in unv_list:
  name_list.append(unv.name)
  hensa_lower_list.append(unv.hensa_lower)
  hensa_upper_list.append(unv.hensa_upper)
  prefecture_list.append(unv.prefecture)
  manage_list.append(unv.manage)
  star_list.append(unv.star)
  review_num_list.append(unv.review_num)
  gakumon_num_list.append(len(unv.gakumon))
  gakubu_num_list.append(len(unv.gakubu))
  rank_list.append(unv.rank)
  try:
    jusoku_rank_list.append(jusoku_dict[unv.name]['jusoku_rank'])
  except KeyError:
    jusoku_rank_list.append('None')
  try:
    jusoku_ritsu_list.append(jusoku_dict[unv.name]['jusoku_ritsu'])
  except KeyError:
    jusoku_ritsu_list.append('None')

# seriesに変換
name_list = pd.Series(name_list)
hensa_lower_list = pd.Series(hensa_lower_list)
hensa_upper_list = pd.Series(hensa_upper_list)
prefecture_list = pd.Series(prefecture_list)
manage_list = pd.Series(manage_list)
star_list = pd.Series(star_list)
review_num_list = pd.Series(review_num_list)
gakumon_num_list = pd.Series(gakumon_num_list)
gakubu_num_list = pd.Series(gakubu_num_list)
rank_list = pd.Series(rank_list)

# DataFrameに格納
unv_all_data = pd.DataFrame({
    'name' : name_list,
    'hensa_lower' : hensa_lower_list,
    'hensa_upper' : hensa_upper_list,
    'prefecture' : prefecture_list,
    'manage' : manage_list,
    'star' : star_list,
    'review_num' : review_num_list,
    # 'review_upper'は一旦保留
    'gakumon_num' : gakumon_num_list,
    'gakubu_num' : gakubu_num_list,
    'rank' : rank_list,
    'jusoku_rank' : jusoku_rank_list,
    'jusoku_ritsu' : jusoku_ritsu_list
})

unv_all_data

,gakubu_num,gakumon_num,hensa_lower,hensa_upper,jusoku_rank,jusoku_ritsu,manage,name,prefecture,rank,review_num,star
0,1,3,35,35,608,94,私立,愛国学園大学,千葉県,None,4,2.75
1,2,2,52.5,65,465,101.9,私立,愛知医科大学,愛知県,115,29,3.92
2,9,18,42.5,50,514,100.2,私立,愛知学院大学,愛知県,231,366,3.77
3,2,7,0,37.5,736,64.7,私立,愛知学泉大学,愛知県,545,56,3.29
4,1,5,47.5,57.5,297,105.7,国立,愛知教育大学,愛知県,50,294,3.85
5,2,6,45,55,459,102.1,公立,愛知県立芸術大学,愛知県,11,41,4.12
6,5,15,50,55,292,105.8,公立,愛知県立大学,愛知県,21,272,4.04
7,1,4,0,35,622,92.4,私立,愛知工科大学,愛知県,553,14,3.25
8,3,9,40,52.5,497,100.6,私立,愛知工業大学,愛知県,407,124,3.58
9,2,8,0,42.5,293,105.8,私立,愛知産業大学,愛知県,77,16,3.99


In [92]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.2MB/s 


In [0]:
unv_all_data.to_csv('unv_all_data.csv')

upload_file = drive.CreateFile()
upload_file.SetContentFile('unv_all_data.csv')
upload_file.Upload()